In [18]:
# Ou est l'iSS
import requests
from datetime import datetime
import smtplib
import time



In [19]:

# Ou est ce que je suis ?
LAT_perso = 2.2740
LONG_perso = 48.5619
EMAIL = "djabril.rezkallah@gmail.com"
PASS = "0000"



In [20]:

def is_iss_overhead():
    # fetching the response from api
    response = requests.get(url="http://api.open-notify.org/iss-now.json")
  
    response.raise_for_status() # Rapporte l'erreur
    
    data = response.json()  # fetching data from api requests

    # Valeurs de l'iss en long et lat dans mes variables
    iss_latitude = float(data["iss_position"]["latitude"])
    iss_longitude = float(data["iss_position"]["longitude"])

    # Incertitude de passage de l'iss à + ou - 10° (choix arbitraire)
    if LAT_perso - 10 <= iss_latitude <= LAT_perso + 10 and LONG_perso - 10 <= iss_longitude <= LONG_perso + 10:
        return True


In [21]:


def is_night():
    parameters = {
        "lat":LAT_perso,
        "long": LONG_perso,
        "format": 0,
    }

    # On va fetcher les données de l'api pour prendre en compte le lever de soleil
    response = requests.get("https://api.sunrise-sunset.org/json", params=parameters)
    response.raise_for_status()
    sun_data = response.json()
    sunrise = int(sun_data["results"]["sunrise"].split("T")[1].split(":")[0])
    sunset = int(sun_data["results"]["sunset"].split("T")[1].split(":")[0])
    # temps actuel
    time_now = datetime.now().hour
    # si on peut l'apercevoir
    if time_now >= sunset or time_now <= sunrise:
        return True


In [22]:


while True:
    
    time.sleep(3)
    # Si il fait nuit j'envois un mail quand l'iss passe
    if is_iss_overhead() and is_night():
        # connecting the gmail server
        connection = smtplib.SMTP("lemailpourenvoyer.gmail.com")
        connection.starttls()
        connection.login(EMAIL, PASS)
        # send the mail
        connection.sendmail(
            from_addr=EMAIL,
            to_addrs=EMAIL,
            msg="Subject: Look Up \n\n The ISS is above you in the sky."
        )

KeyboardInterrupt: ignored